In [3]:
from src import sem_functions, city_functions, expert_functions
import importlib

importlib.reload(sem_functions)

from src.sem_functions import *
from src.city_functions import *
from src.expert_functions import *
import numpy as np
import pandas as pd
import pickle
import gzip
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<module 'src.sem_functions' from '/Users/katharina/Documents/PhD/Scripts_Paper_1/label-uncertainty-so2sat/src/sem_functions.py'>

In [5]:
# import csv 

cities_one_hot_16 = pd.read_csv('/Users/katharina/Documents/PhD/Data/LCZ_Votes/prepared_datasets/cities_one_hot_16.csv').to_numpy()
cities_one_hot_named_16 = pd.read_csv('/Users/katharina/Documents/PhD/Data/LCZ_Votes/prepared_datasets/cities_one_hot_named_16.csv')
all_cities_16 = pd.read_csv('/Users/katharina/Documents/PhD/Data/LCZ_Votes/prepared_datasets/all_cities_16.csv').to_numpy()

## 1. Full Dataset

In [6]:
# run sem on full dataset

sem_16_full_test = sem_fit_var(cities_one_hot_16, K=16, max_iter=20, rtol=1e-3, restarts=5)

iteration 0
Loss: -554587.628046
Loss: -553248.986771
Loss: -552843.053714
better loss on iteration 0: -552843.0537139595
iteration 1
Loss: -554692.837835
Loss: -553292.788831
Loss: -552790.756513
better loss on iteration 1: -552790.7565130633
iteration 2
Loss: -554551.599691
Loss: -553310.056961
Loss: -552883.012412
iteration 3
Loss: -554651.740993
Loss: -553398.922280
Loss: -552939.950125
iteration 4
Loss: -554561.578331
Loss: -553298.511746
Loss: -552790.409610
better loss on iteration 4: -552790.4096098746


In [7]:
# extract parameters

pi_full = sem_16_full_test[1]
theta_full = sem_16_full_test[2]
tau_full = e_step_nozerocorrection(cities_one_hot_16, pi_full, theta_full)

## 2. Per City 

In [8]:
city_list = ['berlin', 'cologne', 'london', 'madrid',
             'milan', 'munich', 'paris', 'rome', 'zurich']

In [9]:
# run sem for instances from each city separately

city_sem_16_original_init_test = [sem_fit_var_per_city(city, data=cities_one_hot_named_16,
                                                  K=16, restarts=5, rtol=1e-3, max_iter=20) for city in city_list]

iteration 0
Loss: -84073.017021
Loss: -83706.938274
Loss: -83578.561115
Loss: -83532.373121
better loss on iteration 0: -83532.3731209939
iteration 1
Loss: -84073.356304
Loss: -83673.551396
Loss: -83568.052255
Loss: -83535.923472
iteration 2
Loss: -84072.797786
Loss: -83673.878739
Loss: -83561.503352
Loss: -83541.392195
iteration 3
Loss: -84070.954638
Loss: -83682.262408
Loss: -83548.056089
Loss: -83529.536435
better loss on iteration 3: -83529.5364352264
iteration 4
Loss: -84075.376038
Loss: -83712.354128
Loss: -83578.787242
Loss: -83536.218665
iteration 0
Loss: -66835.480129
Loss: -66767.268364
Loss: -66731.766451
better loss on iteration 0: -66731.7664511998
iteration 1
Loss: -66833.588116
Loss: -66773.794940
Loss: -66747.299307
iteration 2
Loss: -66833.937521
Loss: -66775.054607
Loss: -66746.998464
iteration 3
Loss: -66834.582164
Loss: -66780.487957
Loss: -66768.647221
iteration 4
Loss: -66834.256982
Loss: -66770.236255
Loss: -66733.860527
iteration 0
Loss: -112893.940234
Loss: -11

## 3. Per Expert

In [10]:
# create expert-specific dataframes: df_1 -> exclude annotations from expert 1

one_hot_experts = expert_data(all_cities_16, cities_one_hot_16)

df  0  done.
df  1  done.
df  2  done.
df  3  done.
df  4  done.
df  5  done.
df  6  done.
df  7  done.
df  8  done.
df  9  done.
df  10  done.


In [11]:
# run sem for each expert-specific dataframe 

sem_16_experts_test = expert_sem(one_hot_experts, K=16, rtol=1e-3, max_iter=20, restarts=5)

exclude expert  0
iteration 0
Loss: -524647.070404
Loss: -524361.375301
better loss on iteration 0: -524361.3753011626
iteration 1
Loss: -524697.396895
Loss: -524263.312790
better loss on iteration 1: -524263.3127901801
iteration 2
Loss: -524713.257947
Loss: -524263.092444
better loss on iteration 2: -524263.0924439398
iteration 3
Loss: -524671.974215
Loss: -524374.079982
iteration 4
Loss: -524668.999285
Loss: -524283.654541
exclude expert  1
iteration 0
Loss: -541865.619397
Loss: -541208.670587
Loss: -540949.627833
better loss on iteration 0: -540949.6278330296
iteration 1
Loss: -541899.051744
Loss: -541294.691388
Loss: -540971.752590
iteration 2
Loss: -541888.560503
Loss: -541231.754116
Loss: -540994.179633
iteration 3
Loss: -541875.533960
Loss: -541201.520341
Loss: -540987.647746
iteration 4
Loss: -541923.094972
Loss: -541263.036412
Loss: -541002.963292
exclude expert  2
iteration 0
Loss: -539097.444851
Loss: -538623.935547
better loss on iteration 0: -538623.9355472099
iteration 1


In [12]:
# save results to lists

pi_experts=[]
theta_experts=[]
tau_experts_org=[]
tau_experts=[]

j=0
for sem in sem_16_experts_test:
    pi_experts.append(sem[1])
    theta_experts.append(sem[2])
    tau_experts_org.append(sem[4])
    tau_experts.append(e_step_nozerocorrection(one_hot_experts[j], sem[1], sem[2]))
    j=j+1

In [14]:
# save results to pickle 

with gzip.open('../data/sem_16_experts_test.pkl', 'wb') as file:
    pickle.dump(sem_16_experts_test, file)

with gzip.open('../data/sem_16_full_test.pkl', 'wb') as file:
    pickle.dump(sem_16_full_test, file)

with gzip.open('../data/city_sem_16_original_init_test.pkl', 'wb') as file:
    pickle.dump(city_sem_16_original_init_test, file)